Path tracking experiment
========================

In [1]:
%matplotlib qt
import visualilzer
import matplotlib.pyplot as plt

fig = plt.figure()
ax0 = fig.add_subplot(2,1,1, projection='polar')
v = visualilzer.Visualizer(ax0)

In [2]:
import pandas as pd
import seaborn as sns
ax1 = fig.add_subplot(2,1,2)
df = pd.DataFrame({"similarity":[], "calc_time":[]})
df.index.name="Step"
df

,similarity,calc_time
Step,,


In [3]:
import numpy as np
import utilf
points = utilf.make_polar_points()
v.set_points(points)


In [ ]:
import importlib
importlib.reload(utilf)
importlib.reload(tf_stepometer)

In [4]:
import tf_stepometer
import time
import os
s = tf_stepometer.Stepometer()
cart_path = np.array([(0,0)])
df = pd.DataFrame({"Similarity":[], "Calc_time":[]})
df.index.name="Step"
ht=0
hp=np.array([0.,0.])

In [5]:
p_,th1_,th2_=None,None,None
for _ in range(100):
    theta =  np.pi/10 * (1+np.random.randn()*0.5)
    # theta = 0
    xy = np.array([-0.25, -0.35]) * (1+np.random.randn(2)*0.5)

    p0 = np.array([utilf.polar_to_cart(p) for p in points])

    p1 = (np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ]) @ p0.T).T + xy

    points = [utilf.cart_to_polar(p) for p in p1]

    v.set_points(points)

    # Calculation
    calctime = time.time()
    # s.theta.assign(np.array([theta]))
    # s.pos.assign(xy)
    
    s.reset()
    # Heuristic set
    p,th1, th2 = utilf.heuristic_pos_theta_theta(p0, p1)
    ht+=th1
    hp += p
    s.pos.assign(hp)
    s.theta.assign(np.array([ht]))
    # cart_path = np.array([(0,0), (1,0)])#(2*np.cos(ht), 2*np.sin(ht))])
    # print(cart_path)
    # cart_path += hp
    cart_path = s(cart_path).numpy()
    cart_path = np.append(cart_path, [(0,0)], axis=0)
    cart_path = s.revert(cart_path).numpy()

    # loss1 = s.calc_loss(p0, p1)
    # s.theta.assign(np.array([th2]))
    # loss2 = s.calc_loss(p0, p1)
    # if sum(loss1) < sum(loss2):
    #     s.theta.assign(np.array([th1]))


    ##
        
    indexes = np.random.randint(0, len(p0), (20, 5))
    # indexes = [list(range(len(p0)))]*20
    # for idx in indexes: s.fit_once(p0[idx],p1)
    calctime = time.time()-calctime
    ##

    # cart_path = np.append(s(cart_path), [(0,0)], axis=0)

    line = [utilf.cart_to_polar(p) for p in s(cart_path)]
    v.set_lines([line])

    a=np.array([*xy,theta])
    b=np.array([*s.pos, *s.theta])
    sim=sum(a*b)/(np.linalg.norm(a)*np.linalg.norm(b))
    df.loc[len(df)]=[sim, calctime]
    ax1.clear()
    sns.lineplot(df.reset_index().iloc[-10:], x='Step', y='Similarity', ax=ax1, markers=True)
    ax1.set_ylim([-1,1.2])
    # fig.savefig(f"tmp{os.sep}{len(os.listdir('tmp'))}")

: 

In [ ]:
df.to_csv(f"tables{os.sep}5pointHeuristicReset.csv")

Heuristic Reset
===============
Use a Heuristic to set an initial position and theta. Hopefully this can allow for better accuracy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

n=150
t=np.random.randn(n)
noise=1
x=3*t + np.random.randn(n)*noise + 0
y=-5*t + np.random.randn(n)*noise + 0
y = y-np.mean(y)
plt.scatter(x,y, s=1)
plt.gca().set_aspect(1.)

In [ ]:
t_=None
t =None
ref_t = 0

In [ ]:
theta=-0.314
pos=[0,0]
p1 = (np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta),  np.cos(theta)]
]) @ np.stack([x,y],-1).T).T + pos
x, y = p1.T

cov=np.cov(np.stack([x,y]))
eval, evec = np.linalg.eig(cov)
# lines = np.stack([np.zeros((2,2)),evec[:,np.argsort(eval)[::-1]].T], axis=-1)
lines = np.stack([np.zeros((2,2)),evec.T], axis=-1)*10
plt.scatter(x,y, s=1, c='k')
plt.plot(*lines[0])
plt.plot(*lines[1])
t_=t
t=np.arctan2(*evec[:,np.argmax(eval)].T[::-1]) + 3*np.pi/4
if t_ is None: t_=t
# print(t, t_)
dels = np.array([t-t_, t-t_+np.pi, t-t_-np.pi])
deltt = dels[np.argmin(np.abs(dels))]
ref_t += deltt
print(deltt)
plt.plot([0, 5*np.cos(ref_t)], [0, 5*np.sin(ref_t)], 'r')
# plt.gca().set_aspect(1.)
plt.xlim(-20,20)
plt.ylim(-20,20)
None

In [ ]:
def heuristic_pos_theta(points):
    '''Takes 2d list of cartesian points'''
    pos = np.sum(points, 0)/len(points)
    cov = np.cov(points.T)
    eval, evec = np.linalg.eig(cov)
    pc0 = evec[:, np.argmax(eval)]
    theta = np.arctan2(*pc0[::-1])
    return (pos, theta)
    

In [ ]:
heuristic_pos_theta(np.stack([x,y],-1))